In [38]:
import chromadb
import os
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
collection_name = os.getenv("collection_name")
db_path = os.getenv("chromadb_path")
client = chromadb.PersistentClient(path=db_path)
collection = client.get_collection(name=collection_name)

In [41]:
keyword = "Police"
results = collection.get(
    # query_texts=keyword,
    # include=["documents"],
    where_document={"$contains": keyword}
)
print(f"total documents found: {len(results)}")

total documents found: 7


In [ ]:
from datetime import datetime
def _date_to_unix(date_str: str) -> int: 
        dt = datetime.strptime(date_str, "%B %d, %Y") 
        
        return int(dt.timestamp())

In [ ]:
start_date = "February 20, 2026"
end_date = "February 21, 2026"
keyword = "Police"


results = collection.get(
    include=["documents"],
    where={
        "$and": [
            {"pub_date": {"$gte": _date_to_unix(start_date)} },
            {"pub_date": {"$lte": _date_to_unix(end_date)} },
            # {"summary": {"$in": [keyword]}}
        ]
    }
)

In [60]:
results

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'included': ['documents'],
 'data': None,
 'metadatas': None}

In [ ]:
results = collection.get(ids=["P2026021600335#chunk=1"])
results["metadatas"][0]

{'summary': "A total lunar eclipse will occur in Hong Kong on March 3 (Tuesday). The event can be directly observed with unaided eyes at places with an unobstructed view to the east, if weather permits. The lunar eclipse will begin before moonrise. The moon will be completely engulfed in the Earth's umbra at 7.04pm, marking the beginning of the total eclipse phase. The maximum eclipse will occur at 7.34pm, and the moon will begin its exit from the umbra at 8.03pm. The total eclipse phase will last for 59 minutes. During the period, the moon will not completely disappear from sight but will appear dull red. This is the result of the blue light component of the sunlight being scattered by the Earth's atmosphere and the remaining red light being refracted onto the lunar surface. The entire process of the lunar eclipse will end at 10.25pm.",
 'pub_date': '2026-02-20',
 'title': 'Total lunar eclipse in Hong Kong on evening of March 3',
 'chunk': 1,
 'pub_time': '15:00:00',
 'news_id': 'P202